In [1]:
import json
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup


OSError: [WinError 126] Le module spécifié est introuvable. Error loading "c:\test\.venv\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
# Charger le dataset
with open('data/data_test.json', 'r') as f:
    data = json.load(f)


In [ ]:
# Séparer les données d'entrée et de sortie
inputs = [d['input'] for d in data]
outputs = [d['output'] for d in data]


In [ ]:
# Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


In [ ]:
# Définir le token de padding
tokenizer.pad_token = tokenizer.eos_token  # Utiliser le token de fin comme token de padding


In [ ]:
# Encoder les entrées et sorties avec padding
max_length = 128
input_ids = [tokenizer.encode(input_text, add_special_tokens=True, max_length=max_length, truncation=True, padding='max_length') for input_text in inputs]
output_ids = [tokenizer.encode(output_text, add_special_tokens=True, max_length=max_length, truncation=True, padding='max_length') for output_text in outputs]


In [ ]:
# Convertir en tenseurs numpy
input_ids = np.array(input_ids).astype(np.int64)  # Convertir en entiers longs
output_ids = np.array(output_ids).astype(np.int64)  # Convertir en entiers longs


In [ ]:
# Charger le modèle pré-entraîné
model = GPT2LMHeadModel.from_pretrained('gpt2')


In [ ]:
# Configurer l'entraînement
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(input_ids) * 10)


In [ ]:
# Configurer l'entraînement
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(input_ids) * 10)


In [ ]:
# Entraîner le modèle
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i in range(len(input_ids)):
        optimizer.zero_grad()
        input_ids_tensor = torch.tensor([input_ids[i]], device=device, dtype=torch.long)  # Convertir en tenseur Long
        output_ids_tensor = torch.tensor([output_ids[i]], device=device, dtype=torch.long)  # Convertir en tenseur Long
        outputs = model(input_ids_tensor, labels=output_ids_tensor)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(input_ids)}')


In [ ]:
# Sauvegarder le modèle
model.save_pretrained('test_generator_model')

In [ ]:
model

## tome 2

In [ ]:
import inspect
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Charger le modèle et le tokenizer
model = GPT2LMHeadModel.from_pretrained('test_generator_model')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Définir la fonction Python à tester
def add():
    return a + b

# Encoder la fonction en entrée
input_text = f"Transform the following Python function into a pytest unit test function:\n{inspect.getsource(add)}"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Créer un attention mask
attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

# Générer le test unitaire avec des paramètres ajustés
output_ids = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=256,
    num_return_sequences=1,
    num_beams=5, 
    temperature=0.7,
    top_k=50,  
    top_p=0.95,  
    early_stopping=True
)

# Décoder et afficher le résultat
generated_test = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_test)

In [ ]:
import json
import numpy as np
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW, get_linear_schedule_with_warmup


In [ ]:
# Charger le dataset
with open('data/data_test.json', 'r') as f:
    data = json.load(f)

# Séparer les données d'entrée et de sortie
inputs = [d['input'] for d in data]
outputs = [d['output'] for d in data]


In [ ]:
# Tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Encoder les entrées et sorties
max_length = 128
input_ids = tokenizer(inputs, max_length=max_length, padding=True, truncation=True, return_tensors='pt').input_ids
output_ids = tokenizer(outputs, max_length=max_length, padding=True, truncation=True, return_tensors='pt').input_ids

# Charger le modèle pré-entraîné
model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [ ]:
# Configurer l'entraînement
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(input_ids) * 10)


In [ ]:
# Entraîner le modèle
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i in range(len(input_ids)):
        optimizer.zero_grad()
        input_ids_tensor = input_ids[i].unsqueeze(0).to(device)  # Convertir en tenseur Long
        output_ids_tensor = output_ids[i].unsqueeze(0).to(device)  # Convertir en tenseur Long
        outputs = model(input_ids_tensor, labels=output_ids_tensor)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(input_ids)}')


In [ ]:

# Sauvegarder le modèle
model.save_pretrained('test_generator_model_t5')

In [ ]:
import inspect
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Charger le modèle et le tokenizer
model = T5ForConditionalGeneration.from_pretrained('test_generator_model_t5')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Définir la fonction Python à tester
def mel2hz(mel):
    return 700 * (10 ** (mel / 2595.0) - 1)

# Encoder la fonction en entrée
input_text = f"Please write a pytest unit test for the following Python function:\n{inspect.getsource(mel2hz)}"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Créer un attention mask
attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

# Générer le test unitaire avec des paramètres ajustés
output_ids = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=256,
    num_return_sequences=1,
    num_beams=5,
    temperature=0.7,  # Ajustement de la température
    top_k=50,
    top_p=0.95,
    early_stopping=True
)

# Décoder et afficher le résultat
generated_test = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_test)